<a href="https://colab.research.google.com/github/snufkin92/colab_tutorial/blob/master/section_12/exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 演習
転移学習もしくはファインチューニングのコードを書いて、カエルの画像と船の画像を識別しましょう。



## 各設定
tensorflowとKerasのバージョンによっては、Kerasのコードでエラーが発生することがあります。エラーを回避するために、以下のセルでKerasのバージョンを指定してインストールします。  
以下のコードは、デフォルトのバージョンでエラーが発生しないときには必要ありません。

In [ ]:
!pip install keras==2.3  # 2020/3/28の時点ではtensorflow2.Xに対応するために必要

なお、Googleの対応により上記のコードは近いうちに必要なくなるかと思います。  
今後tensorflowやKerasのバージョンアップにより同様の問題が発生する可能性がありますが、上記のようにしてバージョンを調整することによる対応が必要になります。  
上記のセルの実行後、**ランタイム→ランタイムを再起動**によりバージョンの更新が完了します。  
念のために、以下のコードによりバージョンを確認しておきましょう。

In [ ]:
import tensorflow
import keras
print(tensorflow.__version__)
print(keras.__version__)

必要なモジュールのインポート、最適化アルゴリズムの設定、及び各定数の設定を行います。  
今回はCIFAR-10の分類を行うので、画像の幅と高さは32、チャンネル数は3に設定します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam

optimizer = Adam()

img_size = 64  # 画像の幅と高さ
n_channel = 3  # チャンネル数
n_mid = 256  # 中間層のニューロン数

batch_size = 32
epochs = 20

## VGG16の導入
ImageNetを使って訓練済みのモデル、VGG16をkeras.applicationsから導入します。  
https://keras.io/ja/applications/#vgg16

In [ ]:
from keras.applications import VGG16

model_vgg16 = VGG16(weights="imagenet",  # ImageNetで学習したパラメータを使用
                 include_top=False,  # 全結合層を含まない
                 input_shape=(img_size, img_size, n_channel))  # 入力の形状
model_vgg16.summary()

## CIFAR-10
Kerasを使い、CIFAR-10を読み込みます。  
今回はこのうちカエルと船の画像のみ使い、画像がカエルか船かを判定できるように訓練します。  
以下のコードでは、CIFAR-10を読み込み、ランダムな25枚の画像を表示します。  
元の画像サイズは32×32なのですが、VGG16の入力は48x48以上のサイズである必要があるため、NumPyのrepeat関数によりサイズを2倍に調整します。  


In [ ]:
from keras.datasets import cifar10

(x_train, t_train), (x_test, t_test) = cifar10.load_data()

# インデックスが6と8のデータのみ取り出す
cifar10_labels = np.array(["airplane", "automobile", "bird", "cat", "deer",
                           "dog", "frog", "horse", "ship", "truck"])
t1, t2 = 6, 8  # frogとship
t_train = t_train.reshape(-1)
t_test = t_test.reshape(-1)
mask_train = np.logical_or(t_train==t1, t_train==t2)  # 6と8のみTrue
x_train = x_train[mask_train]
t_train = t_train[mask_train]
mask_test = np.logical_or(t_test==t1, t_test==t2)  # 6と8のみTrue
x_test = x_test[mask_test]
t_test = t_test[mask_test]

# 画像を拡大
print("Original size:", x_train.shape)
x_train = x_train.repeat(2, axis=1).repeat(2, axis=2)
x_test = x_test.repeat(2, axis=1).repeat(2, axis=2)
print("Input size:", x_train.shape)

n_image = 25
rand_idx = np.random.randint(0, len(x_train), n_image)
plt.figure(figsize=(10,10))  # 画像の表示サイズ
for i in range(n_image):
    cifar_img=plt.subplot(5,5,i+1)
    plt.imshow(x_train[rand_idx[i]])
    label = cifar10_labels[t_train[rand_idx[i]]]
    plt.title(label)
    plt.tick_params(labelbottom=False, labelleft=False, bottom=False, left=False)  # ラベルと目盛りを非表示に

# 正解を0もしくは1に
t_train[t_train==t1] = 0  # frog
t_train[t_train==t2] = 1  # ship
t_test[t_test==t1] = 0  # frog
t_test[t_test==t2] = 1  # ship

## モデルの構築
転移学習、もしくはファインチューニングのモデルを構築しましょう。  
以下のセルの指定した範囲にコードを記述して、 転移学習、もしくはファインチューニングが行われるようにしましょう。

In [ ]:
model = Sequential()
model.add(model_vgg16)

model.add(Flatten())  # 一次元の配列に変換
model.add(Dense(n_mid))
model.add(Activation("relu"))
model.add(Dropout(0.5))  # ドロップアウト
model.add(Dense(1))
model.add(Activation("sigmoid"))

# 以下にコードを追記する
# ---------------- ここから ----------------




# ---------------- ここまで ----------------

model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

## 学習
モデルを訓練します。  
過学習を防ぐために、データ拡張を導入します。  
学習には時間がかかりますので、編集→ノートブックの設定のハードウェアアクセラレーターでGPUを選択しましょう。  

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

x_train = x_train / 255  # 0から1の範囲に収める
x_test = x_test / 255

# データ拡張
generator = ImageDataGenerator(
           rotation_range=0.2,
           width_shift_range=0.2,
           height_shift_range=0.2,
           shear_range=10,
           zoom_range=0.2,
           horizontal_flip=True)
generator.fit(x_train)

# 訓練
history = model.fit_generator(generator.flow(x_train, t_train, batch_size=batch_size),
                              epochs=epochs,
                              validation_data=(x_test, t_test))

## 学習の推移
historyを使って、学習の推移を確認します。

In [ ]:
import matplotlib.pyplot as plt

train_loss = history.history['loss']  # 訓練用データの誤差
train_acc = history.history['accuracy']  # 訓練用データの精度
val_loss = history.history['val_loss']  # 検証用データの誤差
val_acc = history.history['val_accuracy']  # 検証用データの精度

plt.plot(np.arange(len(train_loss)), train_loss, label='loss')
plt.plot(np.arange(len(val_loss)), val_loss, label='val_loss')
plt.legend()
plt.show()

plt.plot(np.arange(len(train_acc)), train_acc, label='acc')
plt.plot(np.arange(len(val_acc)), val_acc, label='val_acc')
plt.legend()
plt.show()

# 解答例
以下は解答例です。  
どうしてもわからない時のみ、参考にしましょう。   

### 転移学習

In [ ]:
model = Sequential()
model.add(model_vgg16)

model.add(Flatten())  # 一次元の配列に変換
model.add(Dense(n_mid))
model.add(Activation("relu"))
model.add(Dropout(0.5))  # ドロップアウト
model.add(Dense(1))
model.add(Activation("sigmoid"))

# 以下にコードを追記する
# ---------------- ここから ----------------
model_vgg16.trainable = False
# ---------------- ここまで ----------------

model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

### ファインチューニング

In [ ]:
model = Sequential()
model.add(model_vgg16)

model.add(Flatten())  # 一次元の配列に変換
model.add(Dense(n_mid))
model.add(Activation("relu"))
model.add(Dropout(0.5))  # ドロップアウト
model.add(Dense(1))
model.add(Activation("sigmoid"))

# 以下にコードを追記する
# ---------------- ここから ----------------
for layer in model_vgg16.layers:
    if layer.name.startswith("block5_conv"):
        layer.trainable = True
    else:
        layer.trainable = False
# ---------------- ここまで ----------------

model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])
model.summary()